In [1]:
import math
import pandas as pd

# ---------------------------------------------------------------------------
# 1. FUNÇÕES DE ANÁLISE
# ---------------------------------------------------------------------------

def calcular_valor_no_default(S0, K, r, T):
    """
    Calcula o preço forward de mercado (F0) e o valor "no-default" (V_0)
    de um contrato forward.
    """
    # Fórmula: F_0 = S_0 * e^(r * T)
    F0 = S0 * math.exp(r * T)
    
    # Fórmula: V_0 = (F_0 - K) * e^(-r * T)
    V_no_default = (F0 - K) * math.exp(-r * T)
    
    return V_no_default, F0

def calcular_probabilidades_default(spreads_dict):
    """
    Calcula as probabilidades de sobrevivência (SP) e default (PD)
    com base em um dicionário de spreads de crédito.
    
    Hipótese: SP(t) = exp(-CS(t) * t)
    """
    sp = {0: 1.0}
    pd_marginal = {}
    
    # DataFrame para armazenar resultados
    dados = []

    for t in sorted(spreads_dict.keys()):
        # Probabilidade de Sobrevivência (Survival Probability)
        sp[t] = math.exp(-spreads_dict[t] * t)
        
        # Probabilidade de Default (PD) marginal no período (t-1, t)
        pd_t = sp[t-1] - sp[t]
        pd_marginal[t] = pd_t
        
        dados.append({
            'Ano': t,
            'Spread': spreads_dict[t],
            'SP(t)': sp[t],
            'PD Marginal (t-1, t)': pd_t
        })
        
    df_prob = pd.DataFrame(dados).set_index('Ano')
    return df_prob, pd_marginal

def analisar_impacto_cva(V_no_default, df_prob, LGD):
    """
    Analisa qualitativamente o impacto do CVA (Credit Valuation Adjustment).
    """
    
    # Verificação 1: Exposição Positiva
    is_exposed = V_no_default > 0
    
    # Verificação 2: Probabilidade de Default
    pd_total_3a = 1.0 - df_prob['SP(t)'].iloc[-1]
    has_pd = pd_total_3a > 0
    
    # Verificação 3: LGD
    has_lgd = LGD > 0
    
    # Conclusão
    if is_exposed and has_pd and has_lgd:
        impacto = "Sim, existe um impacto negativo (CVA > 0)."
        explicacao = (
            f"O contrato tem valor positivo (${V_no_default:,.2f}), "
            f"a contraparte tem {pd_total_3a:.2%} de chance de default em 3 anos, "
            f"e o LGD é de {LGD:.0%}. "
            "O valor real do contrato é (V_no_default - CVA)."
        )
    elif not is_exposed:
        impacto = "Não há impacto de CVA."
        explicacao = (
            "O contrato tem valor negativo ou zero para a empresa. "
            "Não há exposição positiva à contraparte, portanto, não há CVA."
        )
    else:
        impacto = "Não há impacto de CVA."
        explicacao = (
            "A contraparte é livre de risco (PD=0) ou o LGD é 0."
        )
        
    return impacto, explicacao

# ---------------------------------------------------------------------------
# 2. CONFIGURAÇÃO DOS DADOS E PARÂMETROS
# ---------------------------------------------------------------------------

# --- Cenário Base ---
# Parâmetros do Contrato
K_BASE = 1_000_000.0  # Preço de exercício (forward)
S0_BASE = 970_000.0   # Preço spot atual
T_BASE = 3.0          # Vencimento (anos)

# Parâmetros de Mercado
R_LIVRE_RISCO = 0.03        # Taxa livre de risco (contínua)
SIGMA_FWD = 0.25            # Volatilidade do preço forward

# Parâmetros de Crédito da Contraparte C
SPREADS_BASE = {1: 0.02, 2: 0.032, 3: 0.036}
RECUPERACAO = 0.40        # Taxa de recuperação
LGD_BASE = 1.0 - RECUPERACAO   # Perda dado o default (Loss Given Default)

# --- Cenário de Estresse ---
# Aumento de 50% nos spreads de crédito
SPREADS_ESTRESSE = {t: s * 1.5 for t, s in SPREADS_BASE.items()}


# ---------------------------------------------------------------------------
# 3. EXECUÇÃO DA ANÁLISE (CENÁRIO BASE)
# ---------------------------------------------------------------------------

print("="*80)
print(" ANÁLISE DE RISCO DE CRÉDITO DE CONTRATO FORWARD (CENÁRIO BASE)")
print("="*80)

# --- Pergunta 1: Qual o valor "no-default" do contrato? ---
print("\n--- 1. Análise do Valor 'No-Default' (Base) ---\n")
V_no_default_base, F0_base = calcular_valor_no_default(S0_BASE, K_BASE, R_LIVRE_RISCO, T_BASE)

print(f"Preço Spot (S0):                ${S0_BASE:,.2f}")
print(f"Preço do Contrato (K):          ${K_BASE:,.2f}")
print(f"Taxa Livre de Risco (r):        {R_LIVRE_RISCO*100:.1f}%")
print(f"Tempo (T):                      {T_BASE} anos")
print(f"Preço Forward de Mercado (F0):  ${F0_base:,.2f}")
print(f"Valor 'No-Default' (V_0):       ${V_no_default_base:,.2f}")

# --- Pergunta 2: Qual a probabilidade de default da contraparte? ---
print("\n\n--- 2. Análise de Probabilidade de Default (Base) ---\n")
df_prob_base, pd_dict_base = calcular_probabilidades_default(SPREADS_BASE)
print(df_prob_base.to_string(float_format="{:.4f}".format))

# --- Pergunta 3: Qual o impacto do risco de crédito (CVA)? ---
print("\n\n--- 3. Análise do Impacto (CVA) (Base) ---\n")
impacto_base, explicacao_base = analisar_impacto_cva(V_no_default_base, df_prob_base, LGD_BASE)
print(f"Impacto: {impacto_base}")
print(f"Explicação: {explicacao_base}")


# ---------------------------------------------------------------------------
# 4. EXECUÇÃO DA ANÁLISE (CENÁRIO DE ESTRESSE)
# ---------------------------------------------------------------------------

print("\n\n" + "="*80)
print(" ANÁLISE DE ESTRESSE: AUMENTO DE 50% NOS SPREADS DE CRÉDITO")
print("="*80)

# --- Q1 (Estresse): Valor "no-default" ---
# (Não muda, pois S0, K, r, T não mudaram)
V_no_default_estresse = V_no_default_base
print(f"\nValor 'No-Default' (V_0):       ${V_no_default_estresse:,.2f} (Inalterado)")

# --- Q2 (Estresse): Probabilidade de default ---
print("\n\n--- 2. Análise de Probabilidade de Default (Estresse) ---\n")
df_prob_estresse, pd_dict_estresse = calcular_probabilidades_default(SPREADS_ESTRESSE)
print(df_prob_estresse.to_string(float_format="{:.4f}".format))

# --- Q3 (Estresse): Impacto do CVA ---
print("\n\n--- 3. Análise do Impacto (CVA) (Estresse) ---\n")
impacto_estresse, explicacao_estresse = analisar_impacto_cva(V_no_default_estresse, df_prob_estresse, LGD_BASE)
print(f"Impacto: {impacto_estresse}")
print(f"Explicação: {explicacao_estresse}")

# --- Q4: Comparativo do Impacto ---
print("\n\n--- 4. Comparativo do Impacto (Estresse vs. Base) ---\n")
pd_total_base = 1.0 - df_prob_base['SP(t)'].iloc[-1]
pd_total_estresse = 1.0 - df_prob_estresse['SP(t)'].iloc[-1]
print(f"  - Prob. Default Total (Base):      {pd_total_base:.4f} ({pd_total_base:.2%})")
print(f"  - Prob. Default Total (Estresse):  {pd_total_estresse:.4f} ({pd_total_estresse:.2%})")
print(f"  - Aumento na Prob. de Default:   +{(pd_total_estresse/pd_total_base - 1):.1%}")
print("\nConclusão: Um aumento de 50% nos spreads de crédito leva a um "
      f"aumento de {(pd_total_estresse/pd_total_base - 1):.1%} na "
      "probabilidade de default, aumentando o impacto negativo (CVA) no valor do contrato.")

# ---------------------------------------------------------------------------
# 5. HIPÓTESES ASSUMIDAS NA ANÁLISE
# ---------------------------------------------------------------------------

print("\n\n" + "="*80)
print(" HIPÓTESES E PREMISSAS UTILIZADAS NA ANÁLISE")
print("="*80)
print("1. Taxas: As taxas (livre de risco e spreads) são contínuas, "
      "portanto, usamos math.exp() para desconto e capitalização.")
print("2. Probabilidade de Sobrevivência (SP): A SP até o tempo 't' é "
      "aproximada por SP(t) = exp(-CS(t) * t), onde CS(t) é o "
      "credit spread para o vencimento 't'.")
print("3. Análise de CVA: A análise do impacto é qualitativa. Um cálculo "
      "quantitativo do CVA exigiria a Exposição Positiva Esperada (EPE) "
      "em cada ponto de default, o que demandaria uma simulação ou "
      "modelo de precificação (ex: Black-Scholes para EPE de um forward).")
print("4. Cenário de Estresse: O cenário adverso foi definido como um "
      "aumento relativo de 50% nos spreads de crédito da contraparte.")

 ANÁLISE DE RISCO DE CRÉDITO DE CONTRATO FORWARD (CENÁRIO BASE)

--- 1. Análise do Valor 'No-Default' (Base) ---

Preço Spot (S0):                $970,000.00
Preço do Contrato (K):          $1,000,000.00
Taxa Livre de Risco (r):        3.0%
Tempo (T):                      3.0 anos
Preço Forward de Mercado (F0):  $1,061,349.06
Valor 'No-Default' (V_0):       $56,068.81


--- 2. Análise de Probabilidade de Default (Base) ---

     Spread  SP(t)  PD Marginal (t-1, t)
Ano                                     
1    0.0200 0.9802                0.0198
2    0.0320 0.9380                0.0422
3    0.0360 0.8976                0.0404


--- 3. Análise do Impacto (CVA) (Base) ---

Impacto: Sim, existe um impacto negativo (CVA > 0).
Explicação: O contrato tem valor positivo ($56,068.81), a contraparte tem 10.24% de chance de default em 3 anos, e o LGD é de 60%. O valor real do contrato é (V_no_default - CVA).


 ANÁLISE DE ESTRESSE: AUMENTO DE 50% NOS SPREADS DE CRÉDITO

Valor 'No-Default' (V_0): 